In [1]:
from itertools import islice, combinations, product
from pyclustering.cluster.kmedoids import kmedoids
from collections import defaultdict
from scipy.stats import hypergeom
from collections import Counter
from goatools import obo_parser
from functools import partial

import os
import time
import random
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
DATA_DIRECTORY = "/media/clusterduck123/joe/data"
RAW_DATA_DIRECTORY = f"{DATA_DIRECTORY}/raw-data"
YEAST_DIRECTORY = f"{DATA_DIRECTORY}/processed-data/yeast"
NETWORK_DIRECTORY = f"{YEAST_DIRECTORY}/networks"
ANNOTATION_DIRECTORY = f"{YEAST_DIRECTORY}/annotations"
MATRIX_DIRECTORY  = f"{YEAST_DIRECTORY}/distance-matrices"

# Enrichement

#### Load and parse annotation data

In [3]:
aspect  = 'BP'
method  = 'kmedoid'
network = 'systematic_CoEx_COEXPRESdb' 

PPI = nx.read_edgelist(f"{NETWORK_DIRECTORY}/PPI_BioGRID.txt")
annotation_df = pd.read_csv(f"{ANNOTATION_DIRECTORY}/GO_{aspect}_BioGRID-SGD.csv")
go_dag = obo_parser.GODag(f"{RAW_DATA_DIRECTORY}/go-basic.obo")

gene_population = set(PPI.nodes())
GO_population = set(annotation_df.GO_ID)

/media/clusterduck123/joe/data/raw-data/go-basic.obo: fmt(1.2) rel(2019-10-07) 47,285 GO Terms


#### Define convenient dictionaries

In [4]:
# Conversion dictionaries
GO2genes = pd.Series({go_id: set(genes.Systematic_ID) for go_id, genes in annotation_df.groupby('GO_ID')}, 
                     name='nb_genes')

gene2GO  = {gene : set(go_ids.GO_ID) for gene, go_ids in annotation_df.groupby('Systematic_ID')}
global_GO_counter = GO2genes.apply(len)

## Here we GO

### Functions

#### Parser fuctions

In [12]:
def get_number_of_pre_runs(CLUSTER_DIRECTORY, distance, n_clusters = 99):
    splitted_file_names = [name.split('_') for name in os.listdir(CLUSTER_DIRECTORY)]
    pre_runs = [int(run) for run, ncluster in splitted_file_names if ncluster == str(n_clusters)]
    if pre_runs:
        return max(pre_runs)+1
    else:
        return -1

#### Loop functions

In [13]:
def cluster2GO(cluster):
    return set.union(*(gene2GO.get(gene, set()) for gene in cluster))

def is_annotated_in(gene, GO_set):
    return not gene2GO.get(gene,set()).isdisjoint(GO_set)

## GDV

In [14]:
feature = 'GDV'

all_distances = sorted('_'.join(filename.split('_')[:-1]) 
                           for filename in os.listdir(f"{MATRIX_DIRECTORY}/{feature}"))

In [15]:
MIN_CLUSTERS = 2
MAX_CLUSTERS = 100
MAX_RUNS = 10

In [16]:
for distance in {'GDV_similarity', 'canberra', 'mahalanobis'}:
    
    t1 = time.time()
    print(distance)
    
    CLUSTER_DIRECTORY = f"{YEAST_DIRECTORY}/clusterings/{network}/{feature}/{distance}/{method}"
    PVALUE_DIRECTORY  = f"{YEAST_DIRECTORY}/p-values/{network}/{feature}/{distance}/{method}/{aspect}"
    
    if not os.path.exists(PVALUE_DIRECTORY):
        os.makedirs(PVALUE_DIRECTORY)
    
    runs = min(get_number_of_pre_runs(CLUSTER_DIRECTORY, distance, MAX_CLUSTERS), MAX_RUNS)

    for run in range(runs):
        for nb_clusters in range(MIN_CLUSTERS, MAX_CLUSTERS):
            if os.path.exists(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt"):
                continue
            with open(f"{CLUSTER_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt", 'r') as f:
                 cluster_list = [set(line.split()) for line in f]
            cluster_df = pd.Series({gene:cluster_idx 
                                        for cluster_idx,cluster in enumerate(cluster_list) 
                                        for gene in cluster})

            nb_annotated_genes_in_cluster = pd.DataFrame(np.array(
                    [ [len(go_genes & cluster) for cluster in cluster_list] for go_genes in GO2genes]),
                                                       index   = GO2genes.index,
                                                       columns = range(nb_clusters))

            
            k = nb_annotated_genes_in_cluster
            
            K = pd.concat([global_GO_counter[GO2genes.index]]*nb_clusters, axis=1)
            K.columns = k.columns
            
            n = pd.concat([pd.DataFrame(map(len, cluster_list)).T]*len(GO2genes))
            n.index = k.index
            
            N = pd.DataFrame(len(PPI), columns=k.columns, index=k.index)
            
            assert K.eq(k.sum(axis=1), axis=0).all().all()
            assert N.eq(n.sum(axis=1), axis=0).all().all()
            
            # scipy has a really messed up nomeclature... 
            p_values = pd.DataFrame(1-hypergeom.cdf(k=k-1, M=N, N=n, n=K), index=GO2genes.index)
            p_values.to_csv(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt")
            t2 = time.time()
            print(f'{run}_{nb_clusters}: {t2-t1:.2f}sec', end='\r')
    print()

canberra

mahalanobis

GDV_similarity



# GCV-A

In [10]:
feature = 'GCV-A'

all_distances = sorted('_'.join(filename.split('_')[:-1]) 
                           for filename in os.listdir(f"{MATRIX_DIRECTORY}/{feature}"))

In [11]:
MIN_CLUSTERS = 2
MAX_CLUSTERS = 100
MAX_RUNS = 30

In [12]:
method = 'kmedoid'

for distance in {'canberra', 'cityblock'}:
    
    t1 = time.time()
    print(distance)
    
    CLUSTER_DIRECTORY = f"{YEAST_DIRECTORY}/clusterings/{feature}/{distance}/{method}"
    PVALUE_DIRECTORY  = f"{YEAST_DIRECTORY}/p-values/{feature}/{distance}/{method}/{aspect}"
    
    if not os.path.exists(PVALUE_DIRECTORY):
        os.makedirs(PVALUE_DIRECTORY)
    
    runs = min(get_number_of_pre_runs(CLUSTER_DIRECTORY, distance, MAX_CLUSTERS), MAX_RUNS)

    for run in range(runs):
        for nb_clusters in range(MIN_CLUSTERS, MAX_CLUSTERS):
            if os.path.exists(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt"):
                continue
            with open(f"{CLUSTER_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt", 'r') as f:
                 cluster_list = [set(line.split()) for line in f]
            cluster_df = pd.Series({gene:cluster_idx 
                                        for cluster_idx,cluster in enumerate(cluster_list) 
                                        for gene in cluster})

            nb_annotated_genes_in_cluster = pd.DataFrame(np.array(
                    [ [len(go_genes & cluster) for cluster in cluster_list] for go_genes in GO2genes]),
                                                       index   = GO2genes.index,
                                                       columns = range(nb_clusters))

            
            k = nb_annotated_genes_in_cluster
            
            K = pd.concat([global_GO_counter[GO2genes.index]]*nb_clusters, axis=1)
            K.columns = k.columns
            
            n = pd.concat([pd.DataFrame(map(len, cluster_list)).T]*len(GO2genes))
            n.index = k.index
            
            N = pd.DataFrame(len(PPI), columns=k.columns, index=k.index)
            
            assert K.eq(k.sum(axis=1), axis=0).all().all()
            assert N.eq(n.sum(axis=1), axis=0).all().all()
            
            # scipy has a really messed up nomeclature... 
            p_values = pd.DataFrame(1-hypergeom.cdf(k=k-1, M=N, N=n, n=K), index=GO2genes.index)
            p_values.to_csv(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt")
            t2 = time.time()
            print(f'{run}_{nb_clusters}: {t2-t1:.2f}sec', end='\r')
    print()

canberra

cityblock



## GCV-G

In [13]:
feature = 'GCV-G'

all_distances = sorted('_'.join(filename.split('_')[:-1]) 
                           for filename in os.listdir(f"{MATRIX_DIRECTORY}/{feature}"))

In [14]:
MIN_CLUSTERS = 2
MAX_CLUSTERS = 100
MAX_RUNS = 30

In [15]:
method = 'kmedoid'

for distance in {'canberra', 'cityblock'}:
    
    t1 = time.time()
    print(distance)
    
    CLUSTER_DIRECTORY = f"{YEAST_DIRECTORY}/clusterings/{feature}/{distance}/{method}"
    PVALUE_DIRECTORY  = f"{YEAST_DIRECTORY}/p-values/{feature}/{distance}/{method}/{aspect}"
    
    if not os.path.exists(PVALUE_DIRECTORY):
        os.makedirs(PVALUE_DIRECTORY)
    
    runs = min(get_number_of_pre_runs(CLUSTER_DIRECTORY, distance, MAX_CLUSTERS), MAX_RUNS)

    for run in range(runs):
        for nb_clusters in range(MIN_CLUSTERS, MAX_CLUSTERS):
            if os.path.exists(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt"):
                continue
            with open(f"{CLUSTER_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt", 'r') as f:
                 cluster_list = [set(line.split()) for line in f]
            cluster_df = pd.Series({gene:cluster_idx 
                                        for cluster_idx,cluster in enumerate(cluster_list) 
                                        for gene in cluster})

            nb_annotated_genes_in_cluster = pd.DataFrame(np.array(
                    [ [len(go_genes & cluster) for cluster in cluster_list] for go_genes in GO2genes]),
                                                       index   = GO2genes.index,
                                                       columns = range(nb_clusters))

            
            k = nb_annotated_genes_in_cluster
            
            K = pd.concat([global_GO_counter[GO2genes.index]]*nb_clusters, axis=1)
            K.columns = k.columns
            
            n = pd.concat([pd.DataFrame(map(len, cluster_list)).T]*len(GO2genes))
            n.index = k.index
            
            N = pd.DataFrame(len(PPI), columns=k.columns, index=k.index)
            
            assert K.eq(k.sum(axis=1), axis=0).all().all()
            assert N.eq(n.sum(axis=1), axis=0).all().all()
            
            # scipy has a really messed up nomeclature... 
            p_values = pd.DataFrame(1-hypergeom.cdf(k=k-1, M=N, N=n, n=K), index=GO2genes.index)
            p_values.to_csv(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt")
            t2 = time.time()
            print(f'{run}_{nb_clusters}: {t2-t1:.2f}sec', end='\r')
    print()

canberra

cityblock



# GCV-DG

In [16]:
feature = 'GCV-DG'

all_distances = sorted('_'.join(filename.split('_')[:-1]) 
                           for filename in os.listdir(f"{MATRIX_DIRECTORY}/{feature}"))

In [17]:
MIN_CLUSTERS = 2
MAX_CLUSTERS = 100
MAX_RUNS = 10

In [18]:
method = 'kmedoid'

for distance in {'canberra', 'cityblock'}:
    
    t1 = time.time()
    print(distance)
    
    CLUSTER_DIRECTORY = f"{YEAST_DIRECTORY}/clusterings/{feature}/{distance}/{method}"
    PVALUE_DIRECTORY  = f"{YEAST_DIRECTORY}/p-values/{feature}/{distance}/{method}/{aspect}"
    
    if not os.path.exists(PVALUE_DIRECTORY):
        os.makedirs(PVALUE_DIRECTORY)
    
    runs = min(get_number_of_pre_runs(CLUSTER_DIRECTORY, distance, MAX_CLUSTERS), MAX_RUNS)

    for run in range(runs):
        for nb_clusters in range(MIN_CLUSTERS, MAX_CLUSTERS):
            if os.path.exists(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt"):
                continue
            with open(f"{CLUSTER_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt", 'r') as f:
                 cluster_list = [set(line.split()) for line in f]
            cluster_df = pd.Series({gene:cluster_idx 
                                        for cluster_idx,cluster in enumerate(cluster_list) 
                                        for gene in cluster})

            nb_annotated_genes_in_cluster = pd.DataFrame(np.array(
                    [ [len(go_genes & cluster) for cluster in cluster_list] for go_genes in GO2genes]),
                                                       index   = GO2genes.index,
                                                       columns = range(nb_clusters))

            
            k = nb_annotated_genes_in_cluster
            
            K = pd.concat([global_GO_counter[GO2genes.index]]*nb_clusters, axis=1)
            K.columns = k.columns
            
            n = pd.concat([pd.DataFrame(map(len, cluster_list)).T]*len(GO2genes))
            n.index = k.index
            
            N = pd.DataFrame(len(PPI), columns=k.columns, index=k.index)
            
            if not (distance in {'correlation', 'mahalanobis'}):
                assert K.eq(k.sum(axis=1), axis=0).all().all()
                assert N.eq(n.sum(axis=1), axis=0).all().all()
            
            # scipy has a really messed up nomeclature... 
            p_values = pd.DataFrame(1-hypergeom.cdf(k=k-1, M=N, N=n, n=K), index=GO2genes.index)
            p_values.to_csv(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt")
            t2 = time.time()
            print(f'{run}_{nb_clusters}: {t2-t1:.2f}sec', end='\r')
    print()

canberra

cityblock



# GCV-DA

In [19]:
feature = 'GCV-DA'

all_distances = sorted('_'.join(filename.split('_')[:-1]) 
                           for filename in os.listdir(f"{MATRIX_DIRECTORY}/{feature}"))

In [20]:
MIN_CLUSTERS = 2
MAX_CLUSTERS = 100
MAX_RUNS = 30

In [21]:
method = 'kmedoid'

for distance in {'canberra', 'cityblock'}:
    
    t1 = time.time()
    print(distance)
    
    CLUSTER_DIRECTORY = f"{YEAST_DIRECTORY}/clusterings/{feature}/{distance}/{method}"
    PVALUE_DIRECTORY  = f"{YEAST_DIRECTORY}/p-values/{feature}/{distance}/{method}/{aspect}"
    
    if not os.path.exists(PVALUE_DIRECTORY):
        os.makedirs(PVALUE_DIRECTORY)
    
    runs = min(get_number_of_pre_runs(CLUSTER_DIRECTORY, distance, MAX_CLUSTERS), MAX_RUNS)

    for run in range(runs):
        for nb_clusters in range(MIN_CLUSTERS, MAX_CLUSTERS):
            if os.path.exists(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt"):
                continue
            with open(f"{CLUSTER_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt", 'r') as f:
                 cluster_list = [set(line.split()) for line in f]
            cluster_df = pd.Series({gene:cluster_idx 
                                        for cluster_idx,cluster in enumerate(cluster_list) 
                                        for gene in cluster})

            nb_annotated_genes_in_cluster = pd.DataFrame(np.array(
                    [ [len(go_genes & cluster) for cluster in cluster_list] for go_genes in GO2genes]),
                                                       index   = GO2genes.index,
                                                       columns = range(nb_clusters))

            
            k = nb_annotated_genes_in_cluster
            
            K = pd.concat([global_GO_counter[GO2genes.index]]*nb_clusters, axis=1)
            K.columns = k.columns
            
            n = pd.concat([pd.DataFrame(map(len, cluster_list)).T]*len(GO2genes))
            n.index = k.index
            
            N = pd.DataFrame(len(PPI), columns=k.columns, index=k.index)
            
            assert K.eq(k.sum(axis=1), axis=0).all().all()
            assert N.eq(n.sum(axis=1), axis=0).all().all()
            
            # scipy has a really messed up nomeclature... 
            p_values = pd.DataFrame(1-hypergeom.cdf(k=k-1, M=N, N=n, n=K), index=GO2genes.index)
            p_values.to_csv(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt")
            t2 = time.time()
            print(f'{run}_{nb_clusters}: {t2-t1:.2f}sec', end='\r')
    print()

canberra


/home/clusterduck123/Desktop/git/supplements/venv/graco/lib/python3.6/site-packages/pandas/core/computation/expressions.py:194: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  op=op_str, alt_op=unsupported[op_str]


26_99: 148.99sec
cityblock
26_99: 142.00sec


## GCV-all

In [22]:
feature = 'GCV-all'

all_distances = sorted('_'.join(filename.split('_')[:-1]) 
                           for filename in os.listdir(f"{MATRIX_DIRECTORY}/{feature}"))

In [23]:
MIN_CLUSTERS = 2
MAX_CLUSTERS = 100
MAX_RUNS = 5

In [24]:
method = 'kmedoid'

for distance in {'canberra', 'cityblock'}:
    
    t1 = time.time()
    print(distance)
    
    CLUSTER_DIRECTORY = f"{YEAST_DIRECTORY}/clusterings/{feature}/{distance}/{method}"
    PVALUE_DIRECTORY  = f"{YEAST_DIRECTORY}/p-values/{feature}/{distance}/{method}/{aspect}"
    
    if not os.path.exists(PVALUE_DIRECTORY):
        os.makedirs(PVALUE_DIRECTORY)
    
    runs = min(get_number_of_pre_runs(CLUSTER_DIRECTORY, distance, MAX_CLUSTERS), MAX_RUNS)

    for run in range(runs):
        for nb_clusters in range(MIN_CLUSTERS, MAX_CLUSTERS):
            if os.path.exists(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt"):
                continue
            with open(f"{CLUSTER_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt", 'r') as f:
                 cluster_list = [set(line.split()) for line in f]
            cluster_df = pd.Series({gene:cluster_idx 
                                        for cluster_idx,cluster in enumerate(cluster_list) 
                                        for gene in cluster})

            nb_annotated_genes_in_cluster = pd.DataFrame(np.array(
                    [ [len(go_genes & cluster) for cluster in cluster_list] for go_genes in GO2genes]),
                                                       index   = GO2genes.index,
                                                       columns = range(nb_clusters))

            
            k = nb_annotated_genes_in_cluster
            
            K = pd.concat([global_GO_counter[GO2genes.index]]*nb_clusters, axis=1)
            K.columns = k.columns
            
            n = pd.concat([pd.DataFrame(map(len, cluster_list)).T]*len(GO2genes))
            n.index = k.index
            
            N = pd.DataFrame(len(PPI), columns=k.columns, index=k.index)
            
            if not distance in {'correlation', 'mahalanobis'}:
                assert K.eq(k.sum(axis=1), axis=0).all().all()
                assert N.eq(n.sum(axis=1), axis=0).all().all()
            
            # scipy has a really messed up nomeclature... 
            p_values = pd.DataFrame(1-hypergeom.cdf(k=k-1, M=N, N=n, n=K), index=GO2genes.index)
            p_values.to_csv(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt")
            t2 = time.time()
            print(f'{run}_{nb_clusters}: {t2-t1:.2f}sec', end='\r')
    print()

canberra

cityblock



## GCV-nonredundant

In [25]:
feature = 'GCV-nonredundant'

all_distances = sorted('_'.join(filename.split('_')[:-1]) 
                           for filename in os.listdir(f"{MATRIX_DIRECTORY}/{feature}"))

In [26]:
MIN_CLUSTERS = 2
MAX_CLUSTERS = 100
MAX_RUNS = 5

In [27]:
method = 'kmedoid'

for distance in {'canberra', 'cityblock'}:
    
    t1 = time.time()
    print(distance)
    
    CLUSTER_DIRECTORY = f"{YEAST_DIRECTORY}/clusterings/{feature}/{distance}/{method}"
    PVALUE_DIRECTORY  = f"{YEAST_DIRECTORY}/p-values/{feature}/{distance}/{method}/{aspect}"
    
    if not os.path.exists(PVALUE_DIRECTORY):
        os.makedirs(PVALUE_DIRECTORY)
    
    runs = min(get_number_of_pre_runs(CLUSTER_DIRECTORY, distance, MAX_CLUSTERS), MAX_RUNS)

    for run in range(runs):
        for nb_clusters in range(MIN_CLUSTERS, MAX_CLUSTERS):
            if os.path.exists(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt"):
                continue
            with open(f"{CLUSTER_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt", 'r') as f:
                 cluster_list = [set(line.split()) for line in f]
            cluster_df = pd.Series({gene:cluster_idx 
                                        for cluster_idx,cluster in enumerate(cluster_list) 
                                        for gene in cluster})

            nb_annotated_genes_in_cluster = pd.DataFrame(np.array(
                    [ [len(go_genes & cluster) for cluster in cluster_list] for go_genes in GO2genes]),
                                                       index   = GO2genes.index,
                                                       columns = range(nb_clusters))

            
            k = nb_annotated_genes_in_cluster
            
            K = pd.concat([global_GO_counter[GO2genes.index]]*nb_clusters, axis=1)
            K.columns = k.columns
            
            n = pd.concat([pd.DataFrame(map(len, cluster_list)).T]*len(GO2genes))
            n.index = k.index
            
            N = pd.DataFrame(len(PPI), columns=k.columns, index=k.index)
            
            if not distance in {'correlation', 'mahalanobis'}:
                assert K.eq(k.sum(axis=1), axis=0).all().all()
                assert N.eq(n.sum(axis=1), axis=0).all().all()
            
            # scipy has a really messed up nomeclature... 
            p_values = pd.DataFrame(1-hypergeom.cdf(k=k-1, M=N, N=n, n=K), index=GO2genes.index)
            p_values.to_csv(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt")
            t2 = time.time()
            print(f'{run}_{nb_clusters}: {t2-t1:.2f}sec', end='\r')
    print()

canberra

cityblock



## GCV-orca

In [28]:
feature = 'GCV-orca'

all_distances = sorted('_'.join(filename.split('_')[:-1]) 
                           for filename in os.listdir(f"{MATRIX_DIRECTORY}/{feature}"))

In [29]:
MIN_CLUSTERS = 2
MAX_CLUSTERS = 100
MAX_RUNS = 10

In [30]:
method = 'kmedoid'

for distance in {'hellinger'}:
    
    t1 = time.time()
    print(distance)
    
    CLUSTER_DIRECTORY = f"{YEAST_DIRECTORY}/clusterings/{feature}/{distance}/{method}"
    PVALUE_DIRECTORY  = f"{YEAST_DIRECTORY}/p-values/{feature}/{distance}/{method}/{aspect}"
    
    if not os.path.exists(PVALUE_DIRECTORY):
        os.makedirs(PVALUE_DIRECTORY)
    
    runs = min(get_number_of_pre_runs(CLUSTER_DIRECTORY, distance, MAX_CLUSTERS), MAX_RUNS)

    for run in range(runs):
        for nb_clusters in range(MIN_CLUSTERS, MAX_CLUSTERS):
            if os.path.exists(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt"):
                continue
            with open(f"{CLUSTER_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt", 'r') as f:
                 cluster_list = [set(line.split()) for line in f]
            cluster_df = pd.Series({gene:cluster_idx 
                                        for cluster_idx,cluster in enumerate(cluster_list) 
                                        for gene in cluster})

            nb_annotated_genes_in_cluster = pd.DataFrame(np.array(
                    [ [len(go_genes & cluster) for cluster in cluster_list] for go_genes in GO2genes]),
                                                       index   = GO2genes.index,
                                                       columns = range(nb_clusters))

            
            k = nb_annotated_genes_in_cluster
            
            K = pd.concat([global_GO_counter[GO2genes.index]]*nb_clusters, axis=1)
            K.columns = k.columns
            
            n = pd.concat([pd.DataFrame(map(len, cluster_list)).T]*len(GO2genes))
            n.index = k.index
            
            N = pd.DataFrame(len(PPI), columns=k.columns, index=k.index)
            
            if not distance in {'correlation', 'mahalanobis'}:
                assert K.eq(k.sum(axis=1), axis=0).all().all()
                assert N.eq(n.sum(axis=1), axis=0).all().all()
            
            # scipy has a really messed up nomeclature... 
            p_values = pd.DataFrame(1-hypergeom.cdf(k=k-1, M=N, N=n, n=K), index=GO2genes.index)
            p_values.to_csv(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt")
            t2 = time.time()
            print(f'{run}_{nb_clusters}: {t2-t1:.2f}sec', end='\r')
    print()

hellinger



## GCV-orca+

In [31]:
feature = 'GCV-orca+'

all_distances = sorted('_'.join(filename.split('_')[:-1]) 
                           for filename in os.listdir(f"{MATRIX_DIRECTORY}/{feature}"))

In [32]:
MIN_CLUSTERS = 2
MAX_CLUSTERS = 100
MAX_RUNS = 10

In [33]:
method = 'kmedoid'

for distance in {'canberra', 'cityblock'}:
    
    t1 = time.time()
    print(distance)
    
    CLUSTER_DIRECTORY = f"{YEAST_DIRECTORY}/clusterings/{feature}/{distance}/{method}"
    PVALUE_DIRECTORY  = f"{YEAST_DIRECTORY}/p-values/{feature}/{distance}/{method}/{aspect}"
    
    if not os.path.exists(PVALUE_DIRECTORY):
        os.makedirs(PVALUE_DIRECTORY)
    
    runs = min(get_number_of_pre_runs(CLUSTER_DIRECTORY, distance, MAX_CLUSTERS), MAX_RUNS)

    for run in range(runs):
        for nb_clusters in range(MIN_CLUSTERS, MAX_CLUSTERS):
            if os.path.exists(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt"):
                continue
            with open(f"{CLUSTER_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt", 'r') as f:
                 cluster_list = [set(line.split()) for line in f]
            cluster_df = pd.Series({gene:cluster_idx 
                                        for cluster_idx,cluster in enumerate(cluster_list) 
                                        for gene in cluster})

            nb_annotated_genes_in_cluster = pd.DataFrame(np.array(
                    [ [len(go_genes & cluster) for cluster in cluster_list] for go_genes in GO2genes]),
                                                       index   = GO2genes.index,
                                                       columns = range(nb_clusters))

            
            k = nb_annotated_genes_in_cluster
            
            K = pd.concat([global_GO_counter[GO2genes.index]]*nb_clusters, axis=1)
            K.columns = k.columns
            
            n = pd.concat([pd.DataFrame(map(len, cluster_list)).T]*len(GO2genes))
            n.index = k.index
            
            N = pd.DataFrame(len(PPI), columns=k.columns, index=k.index)
            
            if not distance in {'correlation', 'mahalanobis'}:
                assert K.eq(k.sum(axis=1), axis=0).all().all()
                assert N.eq(n.sum(axis=1), axis=0).all().all()
            
            # scipy has a really messed up nomeclature... 
            p_values = pd.DataFrame(1-hypergeom.cdf(k=k-1, M=N, N=n, n=K), index=GO2genes.index)
            p_values.to_csv(f"{PVALUE_DIRECTORY}/{run}_{nb_clusters}_BioGRID.txt")
            t2 = time.time()
            print(f'{run}_{nb_clusters}: {t2-t1:.2f}sec', end='\r')
    print()

canberra

cityblock

